## 1. Notation and key variables

We simulate **two populations** (Left and Right), each with **K neurons** arranged on a ring.

- `s(t) ∈ R^{2K}`: network state (activity), concatenated as `[s_L(t), s_R(t)]`
- `s_L(t) ∈ R^{K}`: left ring activity
- `s_R(t) ∈ R^{K}`: right ring activity

At each step we compute:

- **Velocity gains** *(scalars)*:
  - `v_L(t) = 1 - β_vel * v(t)`
  - `v_R(t) = 1 + β_vel * v(t)`

- **Recurrent input** *(vectors in R^K)*:
  - `g_LL = W_LL s_L`, `g_LR = W_LR s_R`
  - `g_RR = W_RR s_R`, `g_RL = W_RL s_L`

- **Pre-activation current** *(vectors in R^K)*:
  - `G_L = v_L * (g_LL + g_LR + FF_global) + landmark_input`
  - `G_R = v_R * (g_RR + g_RL + FF_global) + landmark_input`
  - `G = [G_L, G_R] ∈ R^{2K}`

- **Activation / firing rate** *(ReLU)*:
  - `F = max(G, 0)`

- **Synaptic low-pass update**:
  - `s(t+dt) = s(t) + (F - s(t)) * dt / τ_s`

### Bump center (phase)
The CAN forms a localized “bump” on the ring. We track its center as a **scalar index**:
- `nn_state[t]`: estimated bump center neuron index (0…K-1 in the code below)

This tracked `nn_state` is the **1D path-integrated variable** used for:
- stopping the trial when the bump reaches an `end_phase`
- triggering internal landmark inputs when the bump crosses learned landmark phases


## 2. Algorithms

### 2.1 Initialization (`init_state`)  [BF09-like + small static bump]
Goal: let the ring settle into a stable bump attractor.

**Algorithm**
1. Create a small noisy velocity input (constant over the init block).
2. Add a weak static Gaussian “seed” input at a chosen ring location (just to pick a phase).
3. Integrate the CAN dynamics for `T_init` seconds.
4. Return the final state `s_init ∈ R^{2K}`.

### 2.2 Trial simulation (`run_trial`)  [BF09-like + optional NFJ24 landmark resets]
Goal: simulate one timing/path-integration trial.

**Algorithm**
1. Initialize `s[:,0] = s_init`.
2. Draw one noisy scalar velocity `v` for the whole trial (Weber noise).
3. For each timestep:
   - compute recurrent inputs and pre-activation currents
   - if internal landmarks are enabled:
     - check which landmark phase has been passed
     - if it is the first entry, store entry time
     - compute a time-envelope amplitude `amp(dt)` (Gaussian in time since entry)
     - inject a spatial Gaussian bump centered at that landmark (or slightly shifted)
   - apply ReLU and low-pass update
   - track the bump center via a local-maximum search around the previous center
4. Stop when the bump phase reaches the stopping threshold.

Outputs per trial:
- `nn_state(t)` trajectory
- trial duration / reaction time `RT = (#steps)*dt`
- optional state matrix `s` for debugging/visualization


## 3. Build the CAN network (parameters)

We mirror the reference MATLAB defaults:

- `dt = 1/2000 s`
- `tau_s = 40 ms`
- Mexican-hat kernel parameters as in the reference code
- Asymmetric shifts: `W_LL` shifted by +1, `W_RR` shifted by -1

> We keep `K=364` to match their ring size.  
> When we want to interpret phases in degrees (0–360), we’ll use helper conversions.


## 4. Initialization sanity check: do we get a stable bump?

We run `init_state()` and plot the final activity in both populations.
You should see:
- a localized bump in each ring
- similar shapes, typically with a small constant shift between L and R (expected in this architecture)


## 5. Single-trial trajectories: with vs without internal landmarks (Fig. 4c-style)

We simulate two single trials:
- `landmarkpresent=False`: pure integrator (drift accumulates)
- `landmarkpresent=True`: internal landmark correction (resets/corrections at learned phases)

We define internal landmark phases as in the MATLAB demo:
- phases at 60°, 120°, 180°, 240°, 300° on a 360° cycle

We convert those degrees to ring indices (0..K-1).


## 8. Reproducing Fig. 4d: RT variance across target durations

The paper reports mean and std of reaction time (RT) for multiple base times (0.65s, 1.3s, ...).

In the CAN simulation, **RT** is the time it takes for the bump to reach the stopping phase.
To study variability vs duration, we need to define a family of tasks with different target durations.

A simple way to do this (and keep the dynamics identical) is:
- pick a baseline speed per condition
- **calibrate** a speed scaling so that the mean RT matches the desired base time
- then measure the RT variability (std) across trials at that base time

Below we implement a lightweight calibration:
- for each target base time, we search a scalar `v_base` such that the mean RT (no-landmark condition) is close to that target
- then we use the same `v_base` for the landmark condition to compare variance reduction

This is not the only choice, but it produces the *qualitative* Fig. 4d comparison: variance with internal landmarks < variance without.
